# 6장 학습 관련 기술

- 매개변수의 최적화 방법
- 가중치 매개변수 초깃값
- 하이퍼파라미터 설정방법

### 오버피팅의 대응책

- 가중치 감소
- 드롭아웃

### 매개변수의 갱신 방법
- 확률적 경사 하강법(SGD) 다시 보기
- 모멘텀(Momentum)
- AdaGrad
- Adam

## 확률적 경사하강법(SGD)

In [17]:
class SGD:
    def __init__(self, lr=0.01):
        """lr은 학습률"""
        self.lr = lr
        
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

$$ W \rightarrow W - \eta\frac{\delta L}{\delta W}  $$


W는 갱신할 가중치 매개변수, 우변의 편미분은 W에 대한 손실함수의 기울기, 에타는 학습률이다. SGD에는 단점이 있는데 학습이 비효율적이다.

![Gradient Descent Optimization Algorithms at Long Valley](http://i.imgur.com/2dKCQHh.gif?1)

![Gradient Descent Optimization Algorithms at Saddle Point](http://i.imgur.com/NKsFHJb.gif?1)

![Gradient Descent Optimization Algorithms at Beale's Function](http://i.imgur.com/pD0hWu5.gif?1)

## 모멘텀

$$ 
v \leftarrow \alpha v - \eta \frac{\delta L}{\delta W} \\
W \leftarrow W + v 
$$

V는 물리에서 말하는 속도에 해당한다


참고문헌에서 본 코드로 보면 간단하다
SGD(Vanilla update)
```python
x += - learning_rate * dx 
```

Momentum update
```python
v = mu * v - learning_rate * dx # integrate velocity
x += v # integrate position
```

각각의 매개변수 갱신방법에 대한 그래프 모양은 위의 그림을 참고하도록 하자.


In [18]:
class Momentum:

    """모멘텀 SGD"""

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]

## AdaGrad
신경망 학습에서는 당연하게도 학습률이 중요하다. 너무 작으면 학습시간이 길어지고, 크면 발산하여 올바른 학습을 할 수 없기 때문. 이를 해결하는 간단한 방법은 일괄적으로 낮추는 것.

AdaGrad는 개별 매개변수에 adative하게 학습률을 조정하여 학습을 진행한다

$$ h \leftarrow h + \frac{\delta L}{\delta W} \odot \frac{\delta L}{\delta W} $$
$$ W \leftarrow W + \eta \frac{1}{\sqrt{h}} \frac{\delta L}{\delta W}  $$

h라는 새로운 변수가 나오는데, 기존 기울기 값을 제곱하여 더해줌 (\odot은 행렬의 원소별 곱셈을 의미함), 매개변수를 갱신할때 \frac{1}{\sqrt{h}}를 곱해 학습률을 조정함

즉 AdaGrad는 과거의 기울기를 제곱하여 계속 더해간다. 그래서 학습을 진행할 수록 강도가 약해짐.

RMSProp은 AdaGrad의 단점을 개선한 방법. 먼 과거의 기울기는 서서이 잊고 새로운 기울기 정보를 크게 반영한다. 이를 지수이동평균이라 하는데, 이것 또한 위에 그림이 있으니 참고하도록 하자.

```python
# RMSProp을 코드로 표현
cache = decay_rate * cache + (1 - decay_rate) * dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)
```
참고문헌의 설명에 따르면.. 
> decay_rate는 초모수이고 보통 [0.9, 0.99, 0.999] 중 하나의 값을 취한다. 주목할 점은 += 업데이트는 Adagrad와 동등하지만, cache가 “어디선가 샌다”. 따라서 RMSProp은 여전히 각 웨이트값을 (그것의 과거 그라디언트) 값으로) 조정하여 성분별로 실질 학습속도를 비슷하게 만드는 효과는 갖고 있지만, Adagrad처럼 학습 속도가 단조적으로 줄지는 않는다. 

In [20]:
class AdaGrad:

    """AdaGrad"""

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


class RMSprop:

    """RMSprop"""

    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] *= self.decay_rate
            self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

## Adam

RMSProp과 AdaGrad를 합친것 같은 알고리즘. 다른 문서에서도 이렇게 설명하는 것으로 보인다. 하이퍼파라미터의 '편향 보정'이 진행된다는 것이 특징!



```python
# Adam을 코드로 간단히 표현
m = beta1*m + (1-beta1)*dx
v = beta2*v + (1-beta2)*(dx**2)
x += - learning_rate * m / (np.sqrt(v) + eps)
```
참고문헌의 설명에 따르면..
> 업데이트는 RMSProp의 업데이트 방식과 정확히 같아 보이는데, 그냥 (노이즈가 껴있을 수도 있는) 그라디언트 dx 대신에 “안정화된” 버전인 m이 사용되었다는 점이 다르다. 논문에 따르면 추천되는 초모수값들은 eps = 1e-8, beta1 = 0.9, beta2 = 0.999이다. 실전에서 Adam은 기본 알고리즘으로 추천되고 있고, 가끔은 RMSProp보다 조금 더 잘 하기도 한다. 그러나 SGD+Nesterov Momentum도 대안으로 해볼만 하다. Adam 업데이트 절차에는 편향 보정(bias correction) 매커니즘이 반영되어 있는데, 벡터 m,v가 나중에 완벽하게 “워밍업” 되기 전에 (iteration의 처음 몇 스텝에서) 초기화되어 0에 편향되어 있다는 점을 보상하기 위해서이다. 

# 참고하면 좋은 글들
- [Gradient Descent Optimization Algorithms 정리](http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html)
- [CS231n Convolutional Neural Networks for Visual Recognition](http://aikorea.org/cs231n/neural-networks-3/#sgd)